In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP 上的 E2E 机器学习：MLOps 阶段 6：serving：使用定制部署容器快速开始使用 Vertex Explainable AI

这是由 [Brian Kang 和 Siping Hu](https://colab.corp.google.com/drive/1aYERnouogPXqlCHlfDRCff04BMV1JpyE?resourcekey=0-BrkuuARc--pA7CvD5LS-oQ#scrollTo=cuKvd9SrmIQw) 贡献的 Colab 笔记本的更新版本。

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_xai_and_custom_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_xai_and_custom_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_xai_and_custom_server.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端MLOps生产。本教程涵盖了第6阶段：serving: 使用可解释的AI开始进行自定义部署容器。

### 目标

在本教程中，您将学习如何构建一个自定义容器来为`Vertex AI Endpoint`上的PyTorch模型提供服务。您将使用FastAPI Python web服务器框架来创建用于服务二进制文件的HTTP服务器。然后，您将将该容器推送到`Artifact Registry`，部署模型，并进行预测和解释请求。

本教程使用以下Google Cloud ML服务:

- `Vertex AI Prediction`
- `Vertex Explainable AI`
- `Google Artifact Registry`

执行的步骤包括:

- 在本地训练PyTorch表格分类器。
- 在本地测试训练模型。
- 使用FastAPI构建HTTP服务器。
- 创建一个包含训练模型和FastAPI服务器的自定义服务容器。
- 在本地测试自定义服务容器。
- 将自定义服务容器推送到Artifact Registry。
- 将自定义服务容器上传为`Model`资源。
- 部署`Model`资源到`Endpoint`资源。
- 向部署的自定义服务容器发送预测请求。
- 向部署的自定义服务容器发出解释请求。

### 数据集

本教程使用的数据集是来自[Scikit-Learn Datasets](https://scikit-learn.org/stable/datasets/)的[Iris数据集](https://scikit-learn.org/stable/datasets/index.html#iris-dataset)。该数据集不需要任何特征工程。训练好的模型可以预测三种不同种类的鸢尾花品种：山鸢尾（setosa）、维吉尼亚鸢尾（virginica）或者变色鸢尾（versicolor）。

### 费用

本教程使用 Google Cloud 中可计费的组件：

* Vertex AI

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

搭建本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已满足运行这个笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Docker
* Git
* Google Cloud SDK（gcloud）
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud的Python开发环境设置指南和Jupyter安装指南提供了满足这些要求的详细说明。以下步骤提供了简明的说明：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口的命令行中运行 `pip install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装执行此笔记本所需的包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install joblib {USER_FLAG} -q
! pip3 install numpy {USER_FLAG} -q
! pip3 install scikit-learn {USER_FLAG} -q
! pip3 install torch {USER_FLAG} -q
! pip3 install "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 {USER_FLAG} -q
! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install --upgrade google-cloud-storage {USER_FLAG} -q
! pip3 install --upgrade python-tabulate $USER_FLAG -q

### 重新启动内核

在安装额外的包之后，您需要重新启动笔记本内核，这样它才能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，可用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter在带有`!`或`%`前缀的行上运行，它会将Python变量插入到这些命令中。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您也可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是 Vertex AI 支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您不能在 Vertex AI 上使用多区域存储桶进行训练。并非所有区域都支持所有 Vertex AI 服务。

了解更多关于[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您在进行直播教程会话，您可能会使用共享的测试帐户或项目。为避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源的名称中。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用Vertex AI Workbench Notebooks**，您的环境已经通过身份验证。

**如果您正在使用Colab**，运行下面的单元格，并在提示时按照说明进行oAuth身份验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **点击创建服务账户**。

2. 在**服务账户名称**字段中输入一个名称，然后点击**创建**。

3. 在**授予该服务账户项目访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex AI”，并选中**Vertex AI管理员**。在过滤框中输入“存储对象管理员”，并选中**存储对象管理员**。

4. 点击创建。一个包含您密钥的JSON文件将下载到本地环境。

5. 在下面的单元格中将您的服务账户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您的笔记本环境如何，都需要执行以下步骤。**

在初始化用于Python的Vertex SDK时，您需要指定一个云存储分期桶。分期桶是您的数据集和模型资源在会话之间保留的地方。

请在下面设置您的云存储桶的名称。桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常数

In [ ]:
import google.cloud.aiplatform as aiplatform

### 初始化 Vertex AI SDK for Python

为您的项目和相应的存储桶初始化 Vertex AI SDK for Python。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 配置项目和资源名称

`IMAGE` - 将要推送的容器图像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

In [ ]:
IMAGE = "xai-fastapi-server"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "xai-fastapi-custom-container"  # @param {type:"string"}

### 启用制品注册表 API

首先，您必须为您的项目启用制品注册表 API 服务。

了解更多关于 [启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

### 创建一个私有的 Docker 仓库

您的第一步是在 Google Artifact Registry 中创建您自己的 Docker 仓库。

1. 运行 `gcloud artifacts repositories create` 命令，使用您的地区和描述“docker 仓库”来创建一个新的 Docker 仓库。

2. 运行 `gcloud artifacts repositories list` 命令来验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置对您的私有存储库的身份验证

在推送或拉取容器镜像之前，配置Docker使用`gcloud`命令行工具来对您所在地区的`Artifact Registry`进行身份验证。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

## 训练模型

接下来，您将为模型创建训练脚本，然后在本地训练模型。

### 脚本

您将创建以下脚本：

- `data.py`：返回经过预处理的训练数据。
- `model.py`：返回用于训练的模型架构。
- `train.py`：返回训练好的模型。
- `server.py`：创建模型服务器。
- `main.py`：创建HTTP服务器。

In [ ]:
%mkdir app

### 创建数据预处理脚本

接下来，您创建脚本`data.py`来获取数据集并预处理训练数据。

In [ ]:
%%writefile app/data.py
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_data():
    # Dataset
    iris = load_iris()
    X = iris['data']
    y = iris['target']
    names = iris['target_names']
    feature_names = iris['feature_names']

    # Scale data to have mean 0 and variance 1 
    # which is importance for convergence of the neural network
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Split the data set into training and testing
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=2)
    
    return X_train, X_test, y_train, y_test

在本地测试脚本。

In [ ]:
! python3 app/data.py

### 创建获取模型架构的脚本

接下来，您创建返回用于训练的模型架构的脚本。

In [ ]:
%%writefile app/model.py
# PyTorch
import torch
import torch.nn.functional as F
import torch.nn as nn

# Build model
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x  
    
def get_model(X_train):
    model = Model(X_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn   = nn.CrossEntropyLoss()
    print(model)
    return model, optimizer, loss_fn

在本地测试脚本。

In [ ]:
! python3 app/model.py

### 训练模型

接下来，您要创建用于训练模型的脚本。

In [ ]:
%%writefile app/train.py
import tqdm
import os
import numpy as np
from data import get_data
from model import get_model

X_train, X_test, y_train, y_test = get_data()


import torch
from torch.autograd import Variable

# Train and save the model
EPOCHS  = 100
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()

model, optimizer, loss_fn = get_model(X_train)

loss_list = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

# Save the model to a checkpoint
print('Saving..')
state = {
    'net': model.state_dict(),
}
if not os.path.isdir('app'):
    os.mkdir('app')
torch.save(state, './app/model.pth')

测试本地脚本。这将训练模型并将模型工件存储在`app/model.pth`中。

In [ ]:
! python3 app/train.py

### 创建模型服务器

接下来，创建用于提供模型的脚本。

In [ ]:
%%writefile app/server.py
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import os

from model import Model

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import os

class IrisClassifier:
    def __init__(self, model_artifact):
        self.net = Model(4)
        self.checkpoint = torch.load(model_artifact)
        self.net.load_state_dict(self.checkpoint['net'])
        self.net.eval()
        self.iris_type = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }
        
    def predict(self, features:dict):
        X = [features['sepal_length'], features['sepal_width'], features['petal_length'], features['petal_width']]
        X = torch.tensor(X)
        X  = torch.unsqueeze(X, 0)
        with torch.no_grad():
            output = self.net(X)
            prob, clas =  output.max(1)

        return {'class': self.iris_type[int(clas.cpu().detach().numpy()[0])],
                'probability': float(prob.cpu().detach().numpy()[0])}

在本地测试脚本。

In [ ]:
! python3 app/server.py

### 在本地执行模型服务器测试

接下来，通过实例化模型服务器并进行本地预测请求来测试模型服务器。

In [ ]:
%%writefile app/test.py
import torch
from server import IrisClassifier
model = IrisClassifier('./app/model.pth')

# The features are scaled
pred1 = model.predict(features={"sepal_length": -1.38535265, "sepal_width": 0.32841405,
                                "petal_length": -1.39706395, "petal_width": 1.3154443})
pred2  = model.predict(features={"sepal_length": -1.02184904, "sepal_width": -2.43394714,
                                 "petal_length": -0.14664056, "petal_width": -0.26238682})
print(pred1)
print(pred2)

In [ ]:
! python3 app/test.py

### 构建一个FastAPI HTTP服务器

最后，在部署容器中，您将需要一个HTTP服务器来处理 `predict` 和 `health` 请求。您可以使用FastAPI来构建HTTP服务器。

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request
from starlette.responses import JSONResponse

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from server import *


app = FastAPI()
'''
gcs_client = storage.Client()

with open("model.joblib", 'wb') as model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
    )

#_model = joblib.load("model.joblib")
'''

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {"status": "healthy"}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    print (body)
    
    import os
    print(os.listdir())

    model = IrisClassifier('./model.pth')
    
    instances = body["instances"]
    output = []
    for i in instances:
        output.append(model.predict(i))
        print(model.predict(i))
    #return 'class' and 'probability'
    return JSONResponse({"predictions": output})

### 添加预启动脚本

FastAPI将在启动服务器之前执行此脚本。`PORT`环境变量设置为等于`AIP_HTTP_PORT`，以便在与Vertex AI期望的端口相同的端口上运行FastAPI。

In [ ]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

将测试示例存储以备后用

接下来，您需要创建一个 JSON 文件，用于向模型服务器发送测试预测请求。

了解更多关于[在线预测请求格式的信息](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)。

In [ ]:
%%writefile instances.json
{
    "instances": [{
        "sepal_length": -1.38535265,
        "sepal_width": 0.32841405,
        "petal_length": -1.39706395,
        "petal_width": 1.3154443
    },{
        "sepal_length": -1.02184904,
        "sepal_width": -2.43394714,
        "petal_length": -0.14664056,
        "petal_width": -0.26238682
    }]
}

## 构建并推送容器到工件注册表

接下来，您将构建自定义部署容器。

### 创建要求文件

首先，创建用于所需安装软件包的requirements.txt文件。

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
torch~=1.11.0

构建自己的容器

编写 Dockerfile，以 `tiangolo/uvicorn-gunicorn-fastapi` 作为基础镜像。这将自动使用 Gunicorn 和 Uvicorn 为您运行 FastAPI。访问[FastAPI文档了解更多关于如何使用Docker部署FastAPI](https://fastapi.tiangolo.com/deployment/docker/)。

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

构建并标记部署镜像。

In [ ]:
DEPLOY_IMAGE = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE}"

if not IS_COLAB:
    ! docker build -t $DEPLOY_IMAGE .
else:
    # install docker daemon
    ! apt-get -qq install docker.io

### 在本地运行和测试容器（可选）

以分离模式在本地运行容器，并提供容器所需的环境变量。这些环境变量将在部署后由 Vertex Prediction 提供给容器。测试 `/health` 和 `/predict` 路由，然后停止运行的镜像。

In [ ]:
if not IS_COLAB:

    ! docker stop local-iris 2>/dev/null
    ! docker rm local-iris 2>/dev/null
    container_id = ! docker run -d -p 80:8080 \
        --name=local-iris \
        -e AIP_HTTP_PORT=8080 \
        -e AIP_HEALTH_ROUTE=/health \
        -e AIP_PREDICT_ROUTE=/predict \
        -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
        -e GOOGLE_APPLICATION_CREDENTIALS=credentials.json \
        {DEPLOY_IMAGE}

    ! sleep 10

测试健康路由

接下来，测试部署容器的健康路由。

In [ ]:
if not IS_COLAB:
    ! curl localhost/health

显示相应的日志条目

In [ ]:
if not IS_COLAB:
    ! docker logs {container_id[0]}

测试预测路线

接下来，测试部署容器的预测路线。

In [ ]:
if not IS_COLAB:
    ! curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

显示相应的日志条目

In [ ]:
if not IS_COLAB:
    ! docker logs {container_id[0]}

停止Docker镜像

现在您已经在本地测试了Docker镜像，您可以停止Docker镜像的执行。

In [ ]:
if not IS_COLAB:
    ! docker stop local-iris

将容器推送到Artifact Registry

接下来，您将为您的客户容器提供一个名称，以便在提交到Google Artifact Registry时使用。

In [ ]:
if not IS_COLAB:
    ! docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{IMAGE}

在 Colab 中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker build . -t $2
docker push $2
kill $(jobs -p)

## 上传自定义服务容器作为Vertex AI `Model` 资源。

接下来，您将上传您的自定义服务容器作为 `Model` 资源。

### 设置说明配置

首先，您需要指定解释设置，这些设置将作为参数传递给上传自定义服务容器时。

**注意**：在选择要用于特征归因的方法时，请确保该方法与您的模型兼容。 继承梯度和XRAI 仅与 TensorFlow 模型和AutoML 图像模型兼容。 Sampled Shapley 适用于表格模型，但无法在图像模型上工作。

了解更多关于[比较解释方法](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview#compare-methods)。

In [ ]:
XAI = "shapley"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)
print(parameters)

EXPLANATION_METADATA = aiplatform.explain.ExplanationMetadata(
    inputs={
        "sepal_length": {},
        "sepal_width": {},
        "petal_length": {},
        "petal_width": {},
    },
    outputs={"probability": {}},
)
print(EXPLANATION_METADATA)

### 上传自定义服务容器

接下来，您将使用`upload()`方法将自定义的服务容器上传为`Model`资源，参数如下：

- `display_name`：`Model`资源的人类可读名称。
- `serving_container_image_name`：您的自定义服务容器。
- `explanation_parameters`：用于配置模型预测解释的参数。
- `explanation_metadata`：描述模型输入和输出以供解释的元数据。

*注意：*由于模型嵌入在服务容器中，因此无需指定`artifacts_uri`参数。

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    serving_container_image_uri=f"{DEPLOY_IMAGE}",
    explanation_parameters=parameters,
    explanation_metadata=EXPLANATION_METADATA,
)

部署自定义服务容器到`Endpoint`资源

接下来，使用`deploy()`方法将您的`Model`资源，用于自定义服务容器，部署到一个`Endpoint`资源。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个预测请求

接下来，您可以使用`predict()`方法向部署的自定义服务容器发送一个在线预测请求。

*注意:* 这些示例已经进行过预缩放。

In [ ]:
endpoint.predict(
    instances=[
        {
            "sepal_length": -1.38535265,
            "sepal_width": 0.32841405,
            "petal_length": -1.39706395,
            "petal_width": 1.3154443,
        },
        {
            "sepal_length": -1.02184904,
            "sepal_width": -2.43394714,
            "petal_length": -0.14664056,
            "petal_width": -0.26238682,
        },
    ]
)

发送解释请求

接下来，您可以通过`explain()`方法向部署的定制服务容器发送在线解释请求。

In [ ]:
prediction = endpoint.explain(
    instances=[
        {
            "sepal_length": -1.38535265,
            "sepal_width": 0.32841405,
            "petal_length": -1.39706395,
            "petal_width": 1.3154443,
        },
        {
            "sepal_length": -1.02184904,
            "sepal_width": -2.43394714,
            "petal_length": -0.14664056,
            "petal_width": -0.26238682,
        },
    ]
)

print(prediction)

检查特征归因

接下来，您将查看此特定示例的特征归因。正面归因值意味着特定特征将您的模型预测上升了那么多，负面归因值则相反。

In [ ]:
INSTANCE = {
    "sepal_length": -1.38535265,
    "sepal_width": 0.32841405,
    "petal_length": -1.39706395,
    "petal_width": 1.3154443,
}

from tabulate import tabulate

feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = True

# Undeploy model and delete endpoint
try:
    endpoint.delete(force=True)

    # Delete the model resource
    model.delete()
except Exception as e:
    print(e)

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {DEPLOY_IMAGE}

! rm -rf app

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}